In [ ]:
import pandas as pd
import os
import numpy as np
import scipy as sp
#
import gensim
from smart_open import smart_open
import multiprocessing

In [ ]:
%matplotlib inline

Steps to be performed:

0) import manually annotated set

1) for final chosen annotations - check self-consistency of doubles

1.1) if all self cosistent, select unique set. else make note of inconsistent annotations and use first occurance value

2) import corpus, look-up corpus, frozen model and reinferred model vectors

3) select and construct reduced corpus/lookup-corpus from annotated set, select corresponding reinferred vectors

4) implement (interactive) selection on this for seed fragments:
    issues: 
    
    - which seed fragment
            
    - how far should expansion go

5) explore and quantify behaviour of returned ground truth as function of cosine distance

## 0) Import annotated ground truth

In [ ]:
groundTruthFile='Manual_annotation_TOTAAL_door_Jeroen_en_Susan.xlsx'
groundTruthFileSheet='Resultaat Manuele Annotatie'

In [ ]:
groundTruth = pd.read_excel(groundTruthFile, sheet_name=groundTruthFileSheet)

In [ ]:
#groundTruth['Titel']


In [ ]:
type(groundTruth)

## 1) Check duplicates for self-consistency

In [ ]:
# show duplicate entries to check whether EINDOORDEEL is consistent. It is, so duplicate can be dropped.
pd.concat(g for _, g in groundTruth.groupby("Titel") if len(g) > 1)


In [ ]:
groundTruth.drop_duplicates(['Titel'],keep='first', inplace=True)
groundTruth.reset_index(drop=True,inplace=True)
groundTruth['Titel'][121]


In [ ]:
len(groundTruth)

## 2) Import corpus and reinferred vectors

The following are routines to instantiate the corpus, this should be moved to a separate utilities module

In [ ]:
stopwordsFile = '/Users/eslt0101/Projects/EviDENce/ML/stopwords-nl-stopwords-iso_expanded_gensim_processed.txt'
corpus_dir = '/Users/eslt0101/Data/eScience/EviDENce/Data/NR-Teksts/EviDENce_NR_output_clean/TargetSize150'
corpus_token_file = 'GV_corpus_text_para_clean.txt'
corpus_text_file = 'GV_corpus_text_para_clean.txt'
corpus_filenames_file = 'GV_corpus_text_para_clean_filenames.txt'

full_ctokf = os.path.join(corpus_dir,corpus_token_file)
full_ctf = os.path.join(corpus_dir,corpus_text_file)
full_cff = os.path.join(corpus_dir,corpus_filenames_file) 

In [ ]:
with open(stopwordsFile) as f:
    dutch_stopwords = f.readlines()
    

In [ ]:
dutch_stopwords = [w.rstrip() for  w in dutch_stopwords]


In [ ]:
def read_corpus(corpus_token_file,labeled=False):
    with smart_open(corpus_token_file,'r') as tf:
        for i,text_line in enumerate(tf):
            if labeled :
                #yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(text_line), [fn_line.rstrip()])
                yield gensim.models.doc2vec.TaggedDocument([token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords], [i])
            else:
                yield [token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords]

In [ ]:
def read_corpus_lookup(corpus_filenames_file, corpus_token_file) :
    with smart_open(corpus_filenames_file, 'r') as fnf, smart_open(corpus_token_file,'r') as tf:
        i=0
        for (fn_line,tf_line) in zip(fnf,tf):
            yield ([i],[fn_line.rstrip()],[tf_line])
            i+=1

In [ ]:
corp = list(read_corpus(full_ctokf,labeled=True))

In [ ]:
corp_lookup = list(read_corpus_lookup(full_cff,full_ctokf))

The following loads the frozen trained model

In [ ]:
model = gensim.utils.SaveLoad.load('/Users/eslt0101/Projects/EviDENce/ML/model_default_v50_mc2_e30_freeze.d2v')


load the reinferred vectors for the full corpus

In [ ]:
reinf_corp_medvec= np.load('/Users/eslt0101/Projects/EviDENce/ML/reinferred_corpus_terms_para_150_medvec.npy')

## 3) construct corpus/vector set limited to ground truth

In [ ]:
corp_lookup[0][1]

In [ ]:
groundTruth['Titel'][0]

In [ ]:
with open(full_cff,'r') as fnf:
    corpFileNames=fnf.readlines()

In [ ]:
corpFileNames = [fn.rstrip().split('.')[0] for fn in corpFileNames ]
#corpFileNames = [fn.rstrip() for fn in corpFileNames ]

In [ ]:
groundTruth_inCorpusIds =[]
for j in range(len(groundTruth)):
    #print(j, groundTruth['Titel'][j])
    gtTitel = groundTruth['Titel'][j]
    corpusIndex = [i for i,ent in enumerate(corpFileNames) if ent == gtTitel]
    groundTruth_inCorpusIds.append(corpusIndex[0])



In [ ]:
#groundTruth_inCorpusIds

In [ ]:

gt_corp_lookup = [corp_lookup[i] for i in groundTruth_inCorpusIds]


In [ ]:
gt_corp=[corp[i] for i in groundTruth_inCorpusIds]


In [ ]:
type(reinf_corp_medvec)

In [ ]:
gt_corp_reinf_medvec=reinf_corp_medvec[groundTruth_inCorpusIds]

### Append reinferred vectors to dataframe

In [ ]:
series_reinfVec =pd.Series(np.array([]))
for i in range(len(gt_corp_reinf_medvec)):
    series_reinfVec[i] = gt_corp_reinf_medvec[i]
    


In [ ]:
groundTruthEx = groundTruth

In [ ]:
groundTruthEx.insert(10,'reinfVec',series_reinfVec)

In [ ]:
(groundTruthEx['reinfVec'][0])

In [ ]:
type(groundTruthEx['reinfVec'])

#### Consider distribution of norms for vectors inferred from coprus. Most w/ 1 < norm < 2

In [ ]:
gtNorms=[]
for i in range(len(groundTruthEx)):
    nn=np.linalg.norm(groundTruthEx['reinfVec'][i])
    #print(nn)
    gtNorms.append(nn)


In [ ]:
np.array(gtNorms)
gtNorms = pd.Series(gtNorms)

In [ ]:
groundTruthEx.insert(11,'vecNorm',gtNorms)

In [ ]:
groundTruthEx['vecNorm'].hist(bins=np.arange(0., 5. + 0.1, 0.1),by=groundTruthEx['EINDOORDEEL_janee'])

In [ ]:
gtNorms.values.argmax()



In [ ]:
gtNorms[320]

In [ ]:
groundTruthEx['EINDOORDEEL_janee'][320]

### set up seed fragments and their vectors

In [ ]:
seed_fromCorpusExt = "Een razzia . Dus die ... . 'Ja , wat doen we ? ' . En we lopen over het dak , en toen hoorden we vanaf de graansilo , daar stond een luchtafweergeschut , en daar kregen we vuur van . Toen vlogen de kogels om onze oren . Toen zij we als de sodomieter via de schoorsteen , in dekking naar Dordtselaan 98 gevlucht . En daar kwamen we en ja , wat doen we ? Ja , wij gaan niet naar beneden toe . Dus wij gingen niet naar beneden toe . Wij gingen perse niet naar beneden ."
seed_manual = 'hij sloeg er op dat het kapot ging en schoot met zijn geweer. het was verschrikkelijk om aan te zien. ik verstopte me, maar kon het geluid van de explosies niet uit mijn ooren krijgen. Toen het over was ben ik zo snel ik kon gaan rennen.'


In [ ]:
seed_fromCorpusExt_processed = [token for token in gensim.utils.simple_preprocess(seed_fromCorpusExt,deacc=True) if token not in dutch_stopwords]
seed_manual_processed = [token for token in gensim.utils.simple_preprocess(seed_manual,deacc=True) if token not in dutch_stopwords]

In [ ]:
infVecArr_seed_fromCorpusExt =[]
infVecArr_seed_manual = []

for i in range(100):
    iV_fCE = model.infer_vector(seed_fromCorpusExt_processed)
    iV_m = model.infer_vector(seed_manual_processed)
    
    infVecArr_seed_fromCorpusExt.append(iV_fCE)
    infVecArr_seed_manual.append(iV_m)
    
iV_seed_fromCorpusExt = np.median(np.array(infVecArr_seed_fromCorpusExt), axis=0)
iV_seed_fromCorpusExt_std = np.array(infVecArr_seed_fromCorpusExt).std(axis=0)

iV_seed_manual = np.median(np.array(infVecArr_seed_manual), axis=0)
iV_seed_manual_std = np.array(infVecArr_seed_manual).std(axis=0)



### define distance metrics for vectors

1.) cosine distance

In [ ]:
def cosdistance(v1,v2):
    d = np.array([1. - sp.spatial.distance.cosine(v1,v2)])
    return d

In [ ]:
def cosdistance_corp(vec,corpus_vectors):
    cosd_vec_corp =[]
    for i in range (len(corpus_vectors)):
        cd = cosdistance(vec,corpus_vectors[i])
        cosd_vec_corp.append(cd)
    cosd_vec_corp = np.array(cosd_vec_corp).flatten()
    return cosd_vec_corp



In [ ]:
iV_seed_fromCorpusExt
#type(groundTruthEx['reinfVec'])

In [ ]:
cosd_seed_fromCorpusExt = cosdistance_corp(iV_seed_fromCorpusExt,groundTruthEx['reinfVec'])

In [ ]:
cosd_seed_manual = cosdistance_corp(iV_seed_manual,groundTruthEx['reinfVec'])

In [ ]:
cdtest[0:10]

In [ ]:
cosd_seed_manual[0:10]

In [ ]:
topN = np.argsort(cosd_seed_fromCorpusExt)[::-1]
topNm = np.argsort(cosd_seed_manual)[::-1]

In [ ]:
groundTruthEx.iloc[topNm[0:25],[2,9]]



In [ ]:
gt_corp_lookup[952]

In [ ]:
cosdistance(iVseries[None,:], groundTruthEx['reinfVec'])

In [ ]:
cosdist = np.array([1. - sp.spatial.distance.cosine(med_infarr1,rinfc) for rinfc in reinf_corp_medvec])

#cosdist.max()
b=np.argsort(cosdist)[::-1]
print(b[0:25])
print(cosdist[b[0:25]])
len(b)

In [ ]:
os.listdir()


In [ ]:
gt = pd.read_excel('ManAtt_JWSH_0001-1000.xlsx',sheet_name='Sheet1')

In [ ]:
gt['Titel']


In [ ]:
aa=gt['JW_janee']


In [ ]:
aa=list(aa)

In [ ]:
bb=[ent.lower() for ent in aa]

In [ ]:
bb

In [ ]:
ids=np.array(np.where(bb=='ja'))

In [ ]:
idja = [i for i, ent in enumerate(bb) if ent == 'ja']


In [ ]:
print(idja)

In [ ]:
print(gt['Tekst'][idja])

In [ ]:
unames=list(set(gt['Titel']))

In [ ]:
len(unames)

In [ ]:
unames

In [ ]:
n_entries =[]
for n in unames:
    n_entries.append(([i for i , ent in enumerate(gt['Titel']) if ent == n]))

In [ ]:
n_entries


In [ ]:
aa= [ent for i,ent in enumerate(n_entries) if len(ent) > 1]
n_entries[410,345]

In [ ]:
gt['JW_janee'][n_entries[aa[0]]]